In [0]:
-- 1. Count bronze rows
SELECT COUNT(*) AS bronze_count FROM workspace.default.bronze_taxi;

-- 2. No non-positive distances (expect 0 bad rows)
SELECT SUM(CASE WHEN trip_distance <= 0 THEN 1 ELSE 0 END) AS invalid_distance
FROM workspace.default.bronze_taxi;

-- 3. Verify suspicious rides exist
SELECT * FROM workspace.default.silver_taxi_clean
WHERE suspicious_flag = TRUE
LIMIT 10;

-- 4. Show weekly aggregates
SELECT * FROM workspace.default.silver_weekly_agg
ORDER BY year DESC, week DESC;

-- 5. Show the 3 highest fare rides
SELECT * FROM workspace.default.gold_top3_fares;


vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,suspicious_flag
V2,2025-01-22T09:49:00.000Z,2025-01-22T09:04:00.000Z,2,5.9,29.5,5.0,34.5,-73.94900000000001,40.749,-73.899,40.769,false
V1,2025-01-21T09:48:00.000Z,2025-01-21T09:03:00.000Z,1,5.8,29.0,3.75,32.75,-73.94800000000001,40.748000000000005,-73.898,40.768,false
V3,2025-01-20T09:47:00.000Z,2025-01-20T09:02:00.000Z,4,5.7,28.5,2.5,31.0,-73.947,40.747,-73.89699999999999,40.766999999999996,false


In [0]:
%python
bronze_count = spark.table("workspace.default.bronze_taxi").count()
assert bronze_count > 0, "Bronze table is empty!"

bad_distances = spark.sql("""
SELECT SUM(CASE WHEN trip_distance <= 0 THEN 1 ELSE 0 END) AS cnt
FROM workspace.default.bronze_taxi
""").collect()[0]["cnt"]

assert bad_distances == 0, f"Found {bad_distances} invalid distances"
print("Validation Passed")


Validation Passed
